In [1]:
import kagglehub
from time import time
import numpy as np  
import kagglehub
import faiss
import pickle

In [2]:
d = 300  # Dimensionality
n = 1000  # Number of points per update

In [3]:
class reader():
    def __init__(self,path):
        self.file=open(path,'r')
        self.counter=0
    
    def read_block(self,n):
        L=[]; word=[]
        for j in range(n):
            line=self.file.readline()
            if line=='': break
            T=line.split()
            word.append(T[0])
            vec=(np.array([float(x) for x in T[1:]]))
            L.append(vec)
            self.counter+=1
            if self.counter%100 == 0:
                print(self.counter,end='\r')
        if len(word)==0:
            return None, None
        word=np.array(word)
        Vecs=np.stack(L)
        X = Vecs.astype('float32')
        return word,X

In [4]:
path = kagglehub.dataset_download("thanakomsn/glove6b300dtxt")
glove_path = path + "/glove.6B.300d.txt" 
#glove_path = "shuffled_output.txt"
_reader=reader(glove_path)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/yoavfreund/.cache/kagglehub/datasets/thanakomsn/glove6b300dtxt/versions/1/glove.6B.300d.txt'

In [5]:
%%time
with open('glove_kmeans_model.pkl','rb') as pkl:
    model=pickle.load(pkl)
centroid_points=model.cluster_centers_

CPU times: user 821 ms, sys: 162 ms, total: 984 ms
Wall time: 1.39 s


In [6]:
centroid_points = np.load('streaming_centroids.npy')
centroid_points.shape

(1000, 300)

In [7]:
# Normalize the vectors to unit length (this converts cosine distance to L2 distance)
faiss.normalize_L2(centroid_points)

# Create the FAISS index (L2 distance)
index = faiss.IndexFlatL2(d)

# Add normalized points to the index
index.add(centroid_points)

#for each centroid we maintain a list of the vectors that are closer to it than to any other centroid.
distances_=[]
words_=[]
for i in range(centroid_points.shape[0]):
    distances_.append([])
    words_.append([])


In [8]:
%%time
from collections import defaultdict

while True:
    query_words, query_points = _reader.read_block(n)
    if query_words is None:
        break
    # Query for the nearest neighbor to a set of points
    faiss.normalize_L2(query_points)  # Normalize the query point
    distances, indices = index.search(query_points, k=1)
    # Output: dict mapping value → list of indices where it occurs

    for i in range(indices.shape[0]):
        i_=indices[i,0]
        words_[i_].append(query_words[i])
        distances_[i_].append(distances[i,0])


CPU times: user 2min 48s, sys: 16.4 s, total: 3min 5s
Wall time: 18.9 s


In [9]:
for i in range(len(words_)):
    distances_[i]=np.array(distances_[i])
    words_[i]= np.array(words_[i])
    order=np.argsort(distances_[i])
    distances_[i] = distances_[i][order]
    words_[i]=words_[i][order]

In [12]:
max([words_[i].shape for i in range(len(words_))])

(6769,)

In [17]:
np.set_printoptions(precision=2, suppress=True)
for i in range(1000):
    print(i,words_[i][:5])
    print(distances_[i][:5])

0 ['toumai' 'otzi' 'babou' 'vanth' 'hiway']
[0.   1.16 1.18 1.19 1.2 ]
1 ['conceals' 'belies' 'flaunts' 'obscures' 'carves']
[0.   1.06 1.07 1.07 1.13]
2 ['161.4' '212.8' '163.2' '198.6' '163.1']
[0.   0.69 0.7  0.71 0.71]
3 ['goatfish' 'microhyla' 'tetraodontidae' 'cabezon' 'amphiprion']
[0.   0.91 0.93 0.95 0.96]
4 ['footbal' '------------------------' 'fooball' 'http://www.abanet.org'
 'passings']
[0.   1.2  1.21 1.23 1.24]
5 ['somniferum' 'papaver' 'surinamensis' 'tigrinum' 'fomes']
[0.   0.43 0.8  0.82 0.83]
6 ['well-executed' 'no-budget' 'well-produced' 'half-hearted'
 'straight-faced']
[0.   1.08 1.08 1.12 1.13]
7 ['fachtna' 'ruanaid' 'huberts' 'illtyd' 'jelley']
[0.   1.03 1.09 1.09 1.12]
8 ['drabs' 'dribs' 'redeposited' 'incrementally' 'intels']
[0.   0.4  1.15 1.21 1.22]
9 ['streeterville' "sha'ab" 'eminönü' 'cannaregio' 'kamurocho']
[0.   1.05 1.06 1.07 1.08]
10 ['ragunan' 'i-charki' 'shichahai' 'clairefontaine' 'kalibata']
[0.   1.11 1.21 1.22 1.22]
11 ['fú' 'jiān' 'jiàn' '